<a href="https://colab.research.google.com/github/geersenthil/Topic-Modeling-/blob/main/LDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import required libraries

In [1]:
import nltk 
nltk.download("stopwords")
!pip install pyLDAvis

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
     |████████████████████████████████| 1.7 MB 3.6 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=5b0933ac2d3f32d9c2f82cd77ead1df61097e5c0e66f41f4b40431b62ade2c64
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
Successfully built pyLDAvis


In [69]:
import re
import pandas as pd
import numpy as np
import json
import glob

#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.parsing import strip_tags, strip_numeric, strip_multiple_whitespaces, stem_text, strip_punctuation, remove_stopwords
from gensim.parsing import preprocess_string

#Dataset
from sklearn.datasets import fetch_20newsgroups

#Spacy
import spacy
from nltk.corpus import stopwords

#Visuals
import pyLDAvis
import pyLDAvis.gensim_models


In [3]:
stopwords = stopwords.words("english")

Load Data into Dataframe

In [15]:
news_group = fetch_20newsgroups()

news_group_data = news_group.data
news_group_target_names = news_group.target_names
news_group_target = news_group.target

In [34]:
news_df = pd.DataFrame({'news': news_group_data})

In [48]:
news_data = news_df['news'].sample(100)

Pre-process data

Lemmatization to get base words

In [49]:
def lemmatization(texts, allowed_postags=["NOUN","ADJ","VERB","ADV"]):
  nlp = spacy.load('en_core_web_sm', disable=["parser", "ner"])
  texts_out = []
  for text in texts:
    doc = nlp(text)
    new_text = []
    for token in doc:
        if token.pos_ in allowed_postags:
          new_text.append(token.lemma_)
    final = " ".join(new_text)
    texts_out.append(final)
  return (texts_out)
 

In [50]:

lemmatizated_text = lemmatization(news_data)


/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/usr/local/lib/python3.7/dist-packages/catalogue.py:126: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use sele

In [51]:
print(lemmatizated_text[0:500])


['subject study posting host - tin version line observe article write # tell fascist politic practice end discrimination now fascism call when force integration armed force opposition most american public time tilting windmill hurt more windmill', 'subject pointer xlib post host organization try implement pointer feature multiple window can take input show output simultaneously other display want implement pointer feature would like pointer come window once choose pointer menu one should able see Can give hint how should proceed new reply will greatly appreciate thank duvvuri@cs.odu.edu', 'subject immunization program line posting previous article say all together now know word meet new boss same old boss chorus will fool again Can little side bet one', "subject myths posting host cunixb.cc.columbia.edu organization line misc.activism.progressive co - moderator write f o health myths office visit other thing give government power bargain doctor hospital fee difference litigation enviro

In [52]:
#tokenize
def gen_words(texts):
  final = []
  for text in texts:
    new = gensim.utils.simple_preprocess(text, deacc=True)
    final.append(new)
  return (final)

data_word = gen_words(lemmatizated_text)

In [53]:
print(data_word[0][0:20])

['subject', 'study', 'posting', 'host', 'tin', 'version', 'line', 'observe', 'article', 'write', 'tell', 'fascist', 'politic', 'practice', 'end', 'discrimination', 'now', 'fascism', 'call', 'when']


In [74]:
#id2word

id2word = corpora.Dictionary(data_word)
corpus = []
for text in data_word:
  new = id2word.doc2bow(text)
  corpus.append(new)

#id and frequency of word
print (corpus[0][0:20])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 2), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1)]


In [75]:
word = id2word[[0][:1][0]]
print(word)

american


In [77]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus[:-1], id2word = id2word, num_topics = 20, random_state = 100, update_every=1, chunksize=100, passes = 10, alpha="auto")

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

In [78]:
test_doc = corpus[-1]
vector = lda_model[test_doc]
print(vector)

# sort by match f
def Sort(sub_li):
  sub_li.sort(key = lambda x: x[1])
  sub_li.reverse()
  return(sub_li)
new_vector = Sort(vector)
print(new_vector)



[(1, 0.045874346), (10, 0.10753278), (11, 0.5649328), (12, 0.2254055), (19, 0.043226607)]
[(11, 0.5649328), (12, 0.2254055), (10, 0.10753278), (1, 0.045874346), (19, 0.043226607)]


Visualise Data

In [72]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, mds="mmds", R=10)
vis

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.223422 -0.081650       1        1  21.324081
11    -0.228591  0.022702       2        1  17.371204
13    -0.051490 -0.263023       3        1  11.739614
16    -0.118933 -0.179867       4        1   6.724613
0     -0.004714  0.240426       5        1   5.513012
4     -0.144504  0.124394       6        1   5.309693
8     -0.189690  0.165908       7        1   4.933132
12    -0.127351 -0.098140       8        1   4.642326
19     0.079502 -0.214029       9        1   4.449066
7      0.231159 -0.127967      10        1   3.745489
10     0.071249 -0.115915      11        1   2.638721
14     0.188695  0.143179      12        1   1.859546
17    -0.042889 -0.006328      13        1   1.759628
3      0.122571  0.148364      14        1   1.645538
5     -0.034454  0.114698      15        1   1.562242
2      0.024134  0.084326      16        1   1.496034
15     0.161282 -0.024886      17        1   1.277470
6      0.150379  0.034037      18        1   1.157511
9      0.059960  0.000755      19        1   0.487516
18     0.077106  0.033013      20        1   0.363563, topic_info=           Term       Freq      Total Category  logprob  loglift
84          old  23.000000  23.000000  Default  10.0000  10.0000
23      subject  79.000000  79.000000  Default   9.0000   9.0000
31        write  68.000000  68.000000  Default   8.0000   8.0000
223   insurance  23.000000  23.000000  Default   7.0000   7.0000
70         will  74.000000  74.000000  Default   6.0000   6.0000
...         ...        ...        ...      ...      ...      ...
19      posting   0.202993  24.876961  Topic20  -5.2267   0.8084
1149   internet   0.202904   6.011214  Topic20  -5.2271   2.2283
9          host   0.202514  27.464601  Topic20  -5.2291   0.7071
274        only   0.201962  29.270907  Topic20  -5.2318   0.6407
87         same   0.201951  18.743942  Topic20  -5.2318   1.0864

[440 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
950       1  0.493864  about
950       4  0.123466  about
950       9  0.123466  about
950      10  0.123466  about
950      12  0.123466  about
...     ...       ...    ...
838       8  0.039626   year
838       9  0.039626   year
838      10  0.039626   year
838      11  0.039626   year
838      15  0.079252   year

[1063 rows x 3 columns], R=10, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 12, 14, 17, 1, 5, 9, 13, 20, 8, 11, 15, 18, 4, 6, 3, 16, 7, 10, 19])